In [2]:
!pip install openai

In [511]:
import os
from openai import OpenAI
from pathlib import Path

api_key = "" 
client = OpenAI(api_key=api_key)


In [513]:
client.files.create(
    file=Path("quiz_format.jsonl"),
    purpose="fine-tune",
)

FileObject(id='file-Fj0GyMp1ZrJMYGFnwKVGum33', bytes=19760, created_at=1732091987, filename='quiz_format.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [515]:
response = client.fine_tuning.jobs.create(
    training_file = "file-Fj0GyMp1ZrJMYGFnwKVGum33",
    model = "gpt-4o-2024-08-06",
    hyperparameters={
    "n_epochs":2
    }
)
print(response)

FineTuningJob(id='ftjob-B5imSfQrxFb35oBfa8RaC9CE', created_at=1732091992, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=2, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-mzKEKjPTDzVL4GIR0reinzMC', result_files=[], seed=2138788366, status='validating_files', trained_tokens=None, training_file='file-Fj0GyMp1ZrJMYGFnwKVGum33', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)


In [525]:
response = client.fine_tuning.jobs.retrieve("ftjob-B5imSfQrxFb35oBfa8RaC9CE")

if response.status == "succeeded" :
    print(response.fine_tuned_model)
else:
    print(response)

ft:gpt-4o-2024-08-06:hm::AVafp0Bq


In [527]:
def generate_quiz(topic):
    prompt = f"Create difficult 3-5 question MCQ quiz on the following question: {topic},do not add anything other than the questions"
    response = client.chat.completions.create(
        model="ft:gpt-4o-2024-08-06:hm::AVafp0Bq",
        messages=[{"role": "user", "content": prompt}],
        n=1,
    )
    #print("Tokens used:", response.usage.total_tokens)
    return response.choices[0].message.content
    

def quiz_answers(prompt):
    prompt = f"Answers to the quiz \n{quiz} \n give the answers in this format [A,B,C,D,A]"
    response = client.chat.completions.create(
        model="ft:gpt-4o-2024-08-06:hm::AVafp0Bq",
        messages=[{"role": "user", "content": prompt}],
        n=1,
        temperature=0
    )
    #print("Tokens used:", response.usage.total_tokens)
    return response.choices[0].message.content

def grade_quiz(quiz_answer, user_answer):
    prompt = f"Mark the following quiz answers: \nCorrect answers: {quiz_answer} \nUser's answers: {user_answer} \nScore the user's answers out of 100%, only return the score as a percentage."
    response = client.chat.completions.create(
        model="ft:gpt-4o-2024-08-06:hm::AVafp0Bq",
        messages=[{"role": "user", "content": prompt}],
        n=1,
        temperature=0
    )
    #print("Tokens used:", response.usage.total_tokens)
    return response.choices[0].message.content

def assign_skill_level(quiz_score):
  if quiz_score >= 90:
    return "Advanced"
  elif quiz_score >= 70:
    return "Intermediate"
  else:
    return "Beginner"

def adjust_quiz_difficulty(quiz_score):
  if quiz_score >= 100:
    return "Much more difficult"
  elif quiz_score >= 90:
      return "More difficult"
  elif quiz_score >= 70:
    return "Slightly more difficult"
  elif quiz_score >= 50:
      return "Similar"
  else:
    return "Slightly easier"

def generate_additional_quiz(quiz, new_difficulty, topic,max_tokens=130):
    prompt = f"Using this quiz as a base \n {quiz} \n create a {new_difficulty} quiz on {topic}, \n(MCQ Only, do not give the answers)"
    response = client.chat.completions.create(
        model="ft:gpt-4o-2024-08-06:hm::AVafp0Bq",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        n=1
    )
    #print("Tokens used:", response.usage.total_tokens)
    return response.choices[0].message.content
    

def explain_answers(quiz, quiz_answers, user_answer, max_tokens = 150):
    prompt = f"Explain any question that the user got wrong in this quiz \n {quiz}, \n these are the answers \n {quiz_answers} and this is the user's answers \n {user_answer} \n explain how to get the answers \n ONLY explain incorrect answers"
    response = client.chat.completions.create(
    model="ft:gpt-4o-2024-08-06:hm::AVafp0Bq",
    messages=[{"role":"user","content":prompt}],
    n=1,
    temperature = 0.5
    )
    return response.choices[0].message.content

def respond_to_query(query, max_tokens=100):
    prompt = f"{query}"
    response = client.chat.completions.create(
        model="ft:gpt-4o-2024-08-06:hm::AVafp0Bq",
        messages=[{"role": "user", "content": prompt}],
        max_tokens = max_tokens,
        n=1
    )
    #print("Tokens used:", response.usage.total_tokens)
    return response.choices[0].message.content

def quiz_question_difficulty(quiz):
    prompt = f"Rate the difficulty of each question in this quiz out of 10 \n{quiz} \n follow this format [0.2,1,5,8,10]"
    response = client.chat.completions.create(
        model="ft:gpt-4o-2024-08-06:hm::AVafp0Bq",
        messages=[{"role": "user", "content": prompt}],
        temperature = 0,
        n=1
    )
    #print("Tokens used:", response.usage.total_tokens)
    return response.choices[0].message.content

def get_topic_difficulty(topic):
    prompt = f"Rate the difficulty of this topic: {topic}, 10 being the most difficult, only give a number"
    response = client.chat.completions.create(
        model="ft:gpt-4o-2024-08-06:hm::AVafp0Bq",
        messages=[{"role": "user", "content": prompt}],
        temperature = 0,
        n=1
    )
    #print("Tokens used:", response.usage.total_tokens)
    return response.choices[0].message.content

In [529]:
topic = "python"
quiz = generate_quiz(topic)
print(quiz)

1. What is the output of the following code?
```python
def func(a, b=[]):
    b.append(a)
    return b

print(func(1))
print(func(2))
```
A) [1] [2]
B) [1] [1, 2]
C) [2] [1, 2]
D) [1, 2] [3]
2. Which of the following is a valid way to create a set in Python?
A) {1, 2, 3}
B) set[1, 2, 3]
C) (1, 2, 3)
D) [1, 2, 3]
3. What will be the result of `all([0, 1, 2])`?
A) True
B) False
C) 1
D) SyntaxError


In [531]:
quiz_answer = quiz_answers(quiz)

In [492]:
user_answer = ['','','','','']
response = grade_quiz(quiz_answer,user_answer)
print(response)
quiz_score = float(response.strip('%'))
new_difficulty = adjust_quiz_difficulty(quiz_score)
print(new_difficulty)

60%
Similar


In [494]:
question_difficulty = quiz_question_difficulty(quiz)
print(question_difficulty)

1. [1]
2. [2]
3. [1]
4. [2]
5. [3]


In [466]:
response = explain_answers(quiz,quiz_answers,user_answer)
print(response)

2. The correct answer is c) Tuple. Tuples are immutable, meaning their elements cannot be changed after they are created. Lists, dictionaries, and sets are mutable.

4. The correct answer is a) 4. The expression `num // 2` performs floor division, resulting in 4.0. The `int()` function then converts it to 4.


In [59]:
additional_quiz = generate_additional_quiz(quiz, new_difficulty,topic)
print(additional_quiz)

1. What is 5 + 2?
A) 6
B) 7
C) 8
D) 9
2. What is 6 + 1?
A) 6
B) 7
C) 8
D) 9
3. What is 4 + 3?
A) 6
B) 7
C) 8
D) 9


In [496]:
question_difficulty = quiz_question_difficulty(quiz)
print(question_difficulty)

1. [1]
2. [2]
3. [1]
4. [2]
5. [3]


In [533]:
print(quiz_answer)
topic_difficulty = get_topic_difficulty(topic)
print(topic_difficulty)

[B, A, B]
3
